In [3]:
import geopandas as gpd
import pandas as pd
from spatialentropy import leibovici_entropy, altieri_entropy


categorisation = pd.read_excel('data/categorisation.xlsx')
categorisation.head(4)

,primary tag,secondary tag,L0 category,L1 category
0,amenity,kindergarten,Education,Early education
1,amenity,school,Education,Primary and secondary education
2,amenity,university,Education,Higher education
3,amenity,language_school,Education,Cultural education


In [23]:
# show how many L1 categories there are under each L0 category
categorisation.groupby('L0 category')['L1 category'].nunique()

L0 category
Education                           5
Entertainment, arts and culture     6
Facilities                          1
Financial                           2
Healthcare                          9
Places of worship                   1
Private transportation              4
Public service                      5
Public transportation               3
Shopping                           12
Sports                              4
Sustenance                          4
Waste management                    1
Name: L1 category, dtype: int64

In [24]:
# how many distinct L1 categories are there?
len(categorisation['L1 category'].unique())

53